In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import time
import datetime

In [2]:
!pip install --upgrade gluonnlp
!pip install --upgrade torch transformers gluonnlp
!pip install --upgrade mxnet
!pip install --upgrade kobert-transformers konlpy


In [8]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import csv
import pandas as pd
# 데이터프레임으로 변환
# 본인 경로에 맞게 수정
ad_data= pd.read_csv('/content/drive/MyDrive/naver_blog_crawl_merged_results_AD.csv',encoding="utf-8")
noad_data=pd.read_csv('/content/drive/MyDrive/naver_blog_crawl_merged_results_noAD.csv',encoding="utf-8")

#ad_data.shape
#noad_data.shape

In [4]:
# prompt: ad_data에 새로운 열(label 열)을 만들어서 전부 1로 인코딩하고

ad_data["label"] = 1

ad_data.sample(n=3)

,제목,링크,내용,label
4724,"/3대밀면이어딨노/동네맛집이 부산맛집으로/주차,웨이팅정보",https://blog.naver.com/chojings/223441895903,"요즘 회사에서 점심은 간단히\n병아리콩샐러드를 먹는데,\n너무 라이트하고 좋은데.....",1
3132,부천 곱창전골 맛집 비교불가 깊은 맛 신이조곱창,https://blog.naver.com/buzafama/223454299140,부천지역에 거주하기 시작한지 오래되지 않은\n시점부터 각종 매스컴을 통해 소문으로 ...,1
4388,[부산 송도 ] 송도해수욕장 밥집 아이들이 좋아하는 돈까스 맛집...,https://blog.naver.com/philnami/223449677742,안녕하세요!\n바이린입니다~\n어디 외출을 하려면 가장 먼저 알아보는 게\n교통편 ...,1


In [5]:
noad_data["label"] = 0
noad_data.sample(n=3)

,제목,링크,내용,label
2875,"인간은 왜 사서 고생을 하는가, 밀린 근황",https://blog.naver.com/highonme/223453022542,오늘은 뚝방길 홍차가게에서\n혼자 독서와 티타임을 즐겼다.\n자양역에서 유명한 카페...,0
3414,과천 샤브샤브맛집 봉덕칼국수 샤브버섯칼국수 또간집,https://blog.naver.com/yyalee/223253457832,"안녕하세요, 필기획입니다!\n오늘은 인천의 남동쪽에 위치한\n남동구 현수막 지정게시...",0
3975,블로그포스팅 맛집 광고 정말 중요하죠,https://blog.naver.com/nspenduor/223026434007,블로그포스팅 맛집 광고 정말 중요하죠\n식당을 운영하게 되면서\n식당광고가 정말\n...,0


In [6]:
import pandas as pd
merged_data = pd.concat([ad_data, noad_data])

#결측치 제거 (내용, 문맥에 관련한 전처리)
context = merged_data[['내용']]
df1= context.dropna()
print('record number',len(df1))

print(df1)

# '내용' 열을 리스트로 변환
context_list = df1['내용'].tolist()


#확인용
#print(context_list[0])

#결측치 제거 (제목은 키워드가 많이 들어갈 수 밖에 없다.->임펙트 중요하므로 파악 필요)
title = merged_data[['제목']]
df2= title.dropna()
print('record number',len(df2))

print(df2)

# '제목' 열을 리스트로 변환
title_list = df2['제목'].tolist()


record number 9074
                                                     내용
0     안녕하세요! 오늘은 추운 겨울, 따끈한 곱창전골에 첫 도전해봤어요. 건대에서 유명한...
1     춘천의 석사동 먹자골목에 자리한 이 식당은 찾기 쉽고 주차장도 마련되어 있어 불편함...
2     안녕하세요! 오늘은 제 인생 첫 곱창전골 도전기를 여러분과 공유하고자 해요. 건대에...
3     양양은 현지인들이 추천하는 맛집 투어로 여행을 즐기는 최고의 장소 중 하나입니다.\...
4     안녕하신가요? 오늘은 송파 지역의 숨은 보석, 초이다이닝 송파점에서의 식사 경험을 ...
...                                                 ...
4020  9월 9일 월요일\n#가로수길데이트 #가로수길 #가로수길데이트코스 #신사동데이트\n...
4021  포토로그\n서비스 점검 안내 5월 23일(목) 오전 1시~6시 정기점검이 진행됩니다...
4022  블로그\n서비스 점검 안내 5월 23일(목) 오전 1시~6시 정기점검이 진행됩니다....
4023  3월 마지막과 4월 1,2\n3월을 잘 마무리할 자신이 없어 미루고 미루다 이렇게 ...
4024  초등학교 때 친구들이랑 오랜만에 만나기로 했는데 어디 갈지 못 정하고 있었는데, 면...

[9074 rows x 1 columns]
record number 9084
                                      제목
0     건대맛집 곱도리전골 첫 도전 후기 '77년생 곱도리 전골식당'
1             춘천 주문진게먹고회먹고 - 대게와 킹크랩의 맛집
2     건대에서 맛본 곱창전골의 매력 한번 맛보면 헤어나올 수 없어요
3     양양 맛집 투어: 동호해변 주변에 인정받은 대게 맛집 서핑크랩
4        송리단길맛집 초이다이닝 송파점 기대한만큼 너무 좋은...
...                

In [7]:
# prompt: train set 과 test set 분리

# train, test 데이터 분리
train_context, test_context, train_labels, test_labels = train_test_split(merged_data['내용'], merged_data['label'], random_state=42, test_size=0.2)

# train, validation 데이터 분리
train_context, val_context, train_labels, val_labels = train_test_split(train_context, train_labels, random_state=42, test_size=0.2)




'''확인용print(len(train_context))
print(len(val_context))
print(len(test_context))'''


'확인용print(len(train_context))\nprint(len(val_context))\nprint(len(test_context))'

In [8]:
# train, test 데이터 분리
train_title, test_title, train_labels, test_labels = train_test_split(merged_data['제목'], merged_data['label'], random_state=42, test_size=0.2)

# train, validation 데이터 분리
train_title, val_title, train_labels, val_labels = train_test_split(train_title, train_labels, random_state=42, test_size=0.2)


In [11]:
import re
def preprocess_korean_text(text):
    # 텍스트 정제: 특수 문자 및 한글, 숫자, 공백 이외의 다른 모든 문자가 제거됩니다.
    text = re.sub(r'[^가-힣0-9\s]', '', text)

    #stop_words에 대한 재정의 필요해보임.
    stop_words = set(['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로', '바꾸어서 말하면', '바꾸어서 한다면', '만약', '그렇지않으면', '까악', '툭', '딱', '삐걱거리다', '보드득', '비걱거리다', '꽈당', '응당', '해야한다', '에 가서', '각', '각각', '여러분', '각종', '각자', '제각기', '하도록하다', '와', '과', '그러므로', '그래서', '고로', '한 까닭에', '하기 때문에', '거니와', '이지만', '대하여', '관하여', '관한', '과연', '실로', '아니나다를가', '생각한대로', '진짜로', '한적이있다', '하곤하였다', '하', '하하', '허허', '아하', '거바', '와', '오', '왜', '어째서', '무엇때문에', '어찌', '하겠는가', '무슨', '어디', '어느곳', '더군다나', '하물며', '더욱이는', '어느때', '언제', '야', '이봐', '어이', '여보시오', '흐흐', '흥', '휴', '헉헉', '헐'])


    # 길이가 2 이하인 단어는 제거
    words = text.split()
    words = [word for word in words if len(word) > 1]

    # 불용어 제거
    words = [word for word in words if word not in stop_words]

    # 정제된 텍스트 반환
    return ' '.join(words)


In [12]:
# 전처리된 텍스트를 저장할 리스트
preprocessed_contexts = []

# 리스트의 각 요소에 대해 전처리 함수를 적용하여 저장
for text in context_list:
    preprocessed_text = preprocess_korean_text(text)
    preprocessed_contexts.append(preprocessed_text)

# 리스트의 처음 몇 개의 요소를 출력
print(preprocessed_contexts[:1])  # 예시로 처음 1개의 요소를 출력


['안녕하세요 오늘은 추운 겨울 따끈한 곱창전골에 도전해봤어요 건대에서 유명한 77년생곱도리전골식당에서의 경험 소개할게요 이곳은 매일 11시 30분부터 11시 30분까지 운영하는데 점심특선도 준비되어 있어 점심시간에 방문하기에도 좋아요 식당 입구에는 행운을 상징하는 네잎클로버 그림이 그려져 있어 긍정적인 기대감을 갖게 했어요 대기 시간에는 이름을 적고 노란색 의자에 앉아서 기다리면 된답니다 대표 메뉴가 보이는 곳에 전시되어 있어 메뉴 선택에 도움이 됐어요 입구에 적힌 위로의 말이 마음에 위안을 주었습니다 잘했어 잘했었고 잘할거야 내부는 사람들로 가득 찼지만 웨이팅을 피해 자리를 잡을 있었습니다 원산지 표시를 확인하니 묵은지 대부분의 재료가 국내산이라 믿음직스러웠어요 점심특선 메뉴가 아주 저렴해서 점심에 방문하기에 좋은 곳이란 생각이 들었어요 네이버 사진 리뷰 작성 음료나 볶음밥을 받을 있는 팁도 알게 되었습니다 우리는 곱도리탕 뼈닭 한우대창과 곱창 모둠 보통맛으로 주문했습니다 라면사리와 공깃밥 하이볼과 소주도 함께 주문했죠 기본 반찬으로 나온 단무지와 두부가 단순하면서도 맛있었고 하이볼의 비주얼도 인상적이었습니다 숙주와 팽이버섯이 푸짐하게 올려진 전골은 보글보글 끓는 모습이 매력적이었어요 처음 맛본 곱도리탕은 양념이 배어서 맛있었고 흑미밥이 특히 인상 깊었습니다 볶음밥은 숙주와 팽이버섯을 추가해 먹으니 더욱 맛있었습니다 곱도리탕 경험이었지만 고추장과 간장 베이스의 소스 덕분에 입문하기 좋았고 부드러운 닭고기가 인상적이었습니다 건대 맛집으로 인정받는 이유를 같았어요 50 77년생 곱도리 전골식당 서울특별시 광진구 동일로24길 70 1층화양동 포스팅은 소정의 광고료를 받고 작성 했습니다']


In [13]:
# 전처리된 텍스트를 저장할 리스트
preprocessed_title = []

# 리스트의 각 요소에 대해 전처리 함수를 적용하여 저장
for text in title_list:
    preprocessed_text = preprocess_korean_text(text)
    preprocessed_title.append(preprocessed_text)

# 리스트의 처음 몇 개의 요소를 출력
print(preprocessed_title[:5])  # 예시로 처음 5개의 요소를 출력

['건대맛집 곱도리전골 도전 후기 77년생 곱도리 전골식당', '춘천 주문진게먹고회먹고 대게와 킹크랩의 맛집', '건대에서 맛본 곱창전골의 매력 한번 맛보면 헤어나올 없어요', '양양 맛집 투어 동호해변 주변에 인정받은 대게 맛집 서핑크랩', '송리단길맛집 초이다이닝 송파점 기대한만큼 너무 좋은']


In [14]:
!pip install transformers



In [15]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

# CLS, SEP 토큰 추가
train_context = ["[CLS] " + str(s) + " [SEP]" for s in train_context]
val_context = ["[CLS] " + str(s) + " [SEP]" for s in val_context]
test_context = ["[CLS] " + str(s) + " [SEP]" for s in test_context]

train_tokens = tokenizer.batch_encode_plus(
    train_context, max_length=512, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='pt'
)

val_tokens = tokenizer.batch_encode_plus(
    val_context, max_length=512, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='pt'
)

test_tokens = tokenizer.batch_encode_plus(
    test_context, max_length=512, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='pt'
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

In [16]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

# CLS, SEP 토큰 추가
train_title = ["[CLS] " + str(s) + " [SEP]" for s in train_title]
val_title = ["[CLS] " + str(s) + " [SEP]" for s in val_title]
test_title = ["[CLS] " + str(s) + " [SEP]" for s in test_title]

train_tokens = tokenizer.batch_encode_plus(
    train_title, max_length=512, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='pt'
)

val_tokens = tokenizer.batch_encode_plus(
    val_title, max_length=512, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='pt'
)

test_tokens = tokenizer.batch_encode_plus(
    test_title, max_length=512, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='pt'
)


In [17]:
import numpy as np

# 토큰화된 데이터를 텐서로 변환
train_seq = torch.stack([tensor.squeeze() for tensor in train_tokens['input_ids']])
train_mask = torch.stack([tensor.squeeze() for tensor in train_tokens['attention_mask']])
train_y = torch.tensor(train_labels.values)

val_seq = torch.stack([tensor.squeeze() for tensor in val_tokens['input_ids']])
val_mask = torch.stack([tensor.squeeze() for tensor in val_tokens['attention_mask']])
val_y = torch.tensor(val_labels.values)

test_seq = torch.stack([tensor.squeeze() for tensor in test_tokens['input_ids']])
test_mask = torch.stack([tensor.squeeze() for tensor in test_tokens['attention_mask']])
test_y = torch.tensor(test_labels.values)


In [18]:
# 배치 사이즈 설정
batch_size = 32

# 데이터로더 생성
train_dataloader = DataLoader(
    list(zip(train_seq, train_mask, train_y)),
    sampler=RandomSampler(train_seq),
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    list(zip(val_seq, val_mask, val_y)),
    sampler=SequentialSampler(val_seq),
    batch_size=batch_size
)

test_dataloader = DataLoader(
    list(zip(test_seq, test_mask, test_y)),
    sampler=SequentialSampler(test_seq),
    batch_size=batch_size
)

In [20]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [21]:
# prompt: 이제 모델을 가져와야하는데 나는 koBERT 모델로 학습시키고 싶거든.

model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2)
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [22]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):

    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [1]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

NameError: name 'random' is not defined

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))